In [ ]:
import pandas as pd
import networkx as nx
import re


In [ ]:
transcript_map = pd.read_csv('testing.csv')

In [ ]:

def map2nodes(transcript_mcap): #function to take transcript map of events and output a map with the input and output frames/states of each node
    event_map = transcript_map.copy()

    #initation nodes take in from 0 and output their frame based on position
    init = event_map[event_map['type'] == 'init'].copy()
    init['start_frame'] = 0
    init['end_frame'] = (init['pos'] - 1) % 3 + 1
    init['end'] = init['pos']

    #stop nodes take in from their frame based on position and output 0 (reinitation), their drop_prob determines number of terminated ribosomes
    stop = event_map[event_map['type'] == 'stop'].copy()
    stop['start_frame'] = (stop['pos'] - 1) % 3 + 1
    stop['end_frame'] = 0
    stop['end'] = stop['pos']
    
    #shift nodes take in from their frame based on position, and output their frame + shift_number (shift+/-shift_number)
    shift = event_map[event_map['type'].str.contains('shift')].copy()
    shift['shift'] = shift['type'].str.extract(r'shift(?P<shift>[\d+-]*)').astype(int)


    shift['start_frame'] = (shift['pos'] - 1) % 3 + 1
    shift['end'] = shift['pos'] + shift['shift']
    shift['end_frame'] = (shift['end'] - 1) % 3 + 1


    #possibly to be implemented
    ires = event_map[event_map['type'] == 'ires'].copy()
    ires['start_frame'] = -1
    ires['end_frame'] = (ires['pos'] - 1) % 3 + 1
    ires['end'] = ires['pos']

    load = event_map[event_map['type'] == 'load'].copy()
    load['end'] = load['pos']
    load[['start_frame', 'end_frame']] = [[-1,0]]
    end = event_map[event_map['type'] == 'end']
    end_idx = end.index
    end_pos = int(end.loc[end.index,'pos'].iloc[0])
    event_map.loc[init.index, ['start_frame', 'end_frame', 'end']] = init[['start_frame', 'end_frame' , 'end']].values
    event_map.loc[ires.index, ['start_frame', 'end_frame', 'end']] = ires[['start_frame', 'end_frame' , 'end']].values

    event_map.loc[stop.index, ['start_frame', 'end_frame', 'end']] = stop[['start_frame', 'end_frame', 'end']].values
    event_map.loc[shift.index, ['start_frame', 'end_frame', 'end']] = shift[[ 'start_frame', 'end_frame', 'end']].values
    event_map.loc[load.index, ['start_frame', 'end_frame', 'end']] = load[[ 'start_frame', 'end_frame', 'end']].values
    event_map = event_map.drop(index=end_idx)
    max_idx = event_map.index.max()

    event_map.loc[max_idx + 1] = [end_pos, 'end-scan0', 0, 1, 0, -1, end_pos ]
    event_map.loc[max_idx + 2] = [end_pos, 'end-tran1', 0, 1, 1, -1, end_pos ]
    event_map.loc[max_idx + 3] = [end_pos, 'end-tran2', 0, 1, 2, -1, end_pos ]
    event_map.loc[max_idx + 4] = [end_pos, 'end-tran3', 0, 1, 3, -1, end_pos ]
    event_map[['pos', 'end', 'start_frame','end_frame']] = event_map[['pos', 'end', 'start_frame','end_frame']].astype(int)
    #'cont_prob' is the probability that a ribosome will hit a node and not change state. it is what is left over after drop and change probs are included
    event_map['cont_prob'] = 1 - (event_map['prob'] + event_map['drop_prob'])
    
    event_map = event_map.sort_values('pos').reset_index()
    event_map = event_map[['pos', 'end', 'type', 'start_frame', 'end_frame', 'prob', 'drop_prob', 'cont_prob']]
    

    return event_map

In [49]:
def add_eventons(nodemap:pd.DataFrame):

    out = nodemap[(nodemap['type'].isin(['init', 'stop'])) | nodemap['type'].str.contains('shift')].copy()
    keep = nodemap.drop(out.index).drop('end', axis=1)
    out['pos_begin'] = out['pos']
    out['pos_finish'] = out['end']
    out['type_begin'] = out['type'] + '/begin'
    out['type_finish'] = out['type'] + '/finish'
    out['start_frame_begin'] = out['start_frame']
    out['end_frame_begin'] = out['end_frame']
    out['start_frame_finish'] = out['end_frame']
    out['end_frame_finish'] = out['end_frame']
    out[out.filter(regex='prob').columns + '_begin'] = out[out.filter(regex='prob').columns]
    out['prob_finish'] = [None if x == 0 else 1 for x in out['prob']]
    out['drop_prob_finish'] = 0
    out['cont_prob_finish'] = 1
 
    begin = out.filter( regex=r'begin')
    begin.columns = keep.columns
    finish = out.filter(regex=r'finish').dropna()
    finish.columns = keep.columns

    out = pd.concat([keep,begin,finish])

    
    out['node'] = out[['pos', 'start_frame']].apply(tuple, axis =1)
    return out.reset_index().drop('index', axis = 1)

In [50]:
def next_node(pos, frame, prob, node_map: pd.DataFrame, event, cont:bool): 
    #finds the next node that matches frame as long as probability of the edge is greater than 0
    if prob == 0:
        return pd.Series([None, None])
    if 'shift' in event and 'begin' in event and not cont:
        pos = pos + int(re.search(r'(\+|-)\d', event)[0])
    following_events = node_map[
        (node_map['pos'] >= pos) 
        & (node_map['start_frame'] == frame) 
        & ((node_map['type'] != event) | (node_map['pos'] != pos))
    ]
    # print(pos, frame, event)
    # print(following_events)
    if following_events.empty:
        return pd.Series([(0,-1), 'return2bulk'])  # no next node found
    state = None
    
    #determines state of edge (translating/scan0ning) 
    if cont:
        if 'begin' in  event:
            if 'init' in event:
                state = 'scan'
            elif 'shift' in event:
                state = 'tran' + str(int(frame))
            elif 'stop' in event:
                state = 'tran' + str(int(frame))
        elif 'finish' in event:
            if 'init' in event:
                state = 'tran' + str(int(frame))
            elif 'shift' in event:
                state = 'tran' + str(int(frame))
            elif 'stop' in event:
                state = 'scan'
        else:
            state = 'scan'
    else:
        if 'begin' in  event:
            if 'init' in event:
                state = 'init'
            elif 'stop' in event:
                state = 'rein'
            elif 'shift' in event:
                state = 'shift'
        elif 'finish' in event:
            if 'init' in event:
                state = 'tran' + str(int(frame))
            elif 'shift' in event:
                state = 'tran' + str(int(frame))
            elif 'stop' in event:
                state = 'scan'
        elif event == 'ires':
            state = 'tran' + str(int(frame))
        elif event == 'load':
            state = 'scan'


    first_event = following_events.loc[following_events['pos'].idxmin()]
    return pd.Series([first_event['node'], state]) 

In [ ]:
def ribograph(node_map: pd.DataFrame):
    
    #creates an edge list of all nodes connected to the loading node based on their positions and frames
    graph = node_map.copy()

    # next node using end_frame
    graph[['change','change_type']] = graph.apply(
        lambda row: next_node(row['pos'], row['end_frame'], row['prob'], node_map, row['type'], False),
        axis=1
    )

    print(graph)
    # next node using start_frame
    graph[['cont','cont_type']] = graph.apply(
        lambda row: next_node(row['pos'], row['start_frame'], row['cont_prob'], node_map, row['type'], True),
        axis=1
    )
    print(graph)

    #automatically assigns 'bulk' node to target for all rows with a bulk probability greater than 0
    graph['drop'] = [(-1,-1) if x > 0 else None for x in graph['drop_prob']]
    graph['drop_type'] = 'drop' 
    print(graph)
    
    #creates a 'bulk' node which all drop off feeds into, needed for path search later
    graph.loc[-1] = [-1,'bulk',-1,-1,1,0,0,(-1,-1),None,None,None,None,None, None]    
    load = graph[graph['type'].isin(['load','ires'])]
    truth = graph['node'].isin(graph[['change', 'cont', 'drop']].values.flatten())

    graph = graph[truth]
    print(graph)

    graph = pd.concat([graph, load])
    return graph

In [52]:
def table2edgelist(graph):
        #creates a list of the edges based on the node pairs (could this be done with pivot?)
        change = graph[['node','change', 'prob', 'change_type']].dropna().copy()
        drop = graph[['node','drop', 'drop_prob', 'drop_type']].dropna().copy()
        cont = graph[['node','cont', 'cont_prob', 'cont_type']].dropna().copy()
        change.columns = ['node','target','weight', 'state']
        drop.columns = ['node','target','weight', 'state' ]
        cont.columns = ['node','target','weight', 'state']
        edgelist = pd.concat([change, drop, cont])
        return edgelist.reset_index().drop('index', axis=1).drop_duplicates()


In [53]:
def get_ribopaths(graph:nx.DiGraph, map:pd.DataFrame):
    #finds all ribosome paths
    all_paths = []
    for start in map[map['type'].isin(['load','ires'])]['node']:
        path = nx.all_simple_edge_paths(graph, source=start, target=(-1,-1))
        all_paths.extend(path)
    return(all_paths)

In [54]:
def calc_flux(graph, edgelist, edge_paths):
    #calculates the flux for each path, shoudl probably split thsi into a function for a single path and a 
    out = edgelist.copy()
    out['edge_tuple'] = out[['node', 'target']].apply(tuple, axis =1)
    

    path_number = 0
    for edge_path in edge_paths:
        path_number += 1
        flux = 1
        #iterates through path multiplying probability to calculate final flux at return2bulk
        for edge in edge_path:
            prob = graph[edge[0]][edge[1]]['weight']
            flux = flux * prob
        path_frame = out[out['edge_tuple'].isin(edge_path)].copy().set_index('edge_tuple')
        path_frame['value'] = flux
        out['value' + str(path_number)] = out['edge_tuple'].map(path_frame['value']).fillna(0)
    out['value'] = out[list(out.filter(regex='value'))].sum(axis=1)
    out = out[['node','target','weight','state','value']]
    return out

In [55]:
print('transcript_map')
print(transcript_map)

transcript_map
   pos      type  prob  drop_prob
0    1      load   1.0        0.0
1   20      init   0.5        0.0
2   26  shift-10   0.5        0.0
3   32      init   0.5        0.0
4   60      stop   0.0        1.0
5   80      stop   0.5        0.5
6  100       end   0.0        1.0


In [56]:

nodemap = map2nodes(transcript_map=transcript_map)
print('nodemap')
print(nodemap)

nodemap = add_eventons(nodemap)
print('move_events')
print(nodemap)

nodemap
   pos  end       type  start_frame  end_frame  prob  drop_prob  cont_prob
0    1    1       load           -1          0   1.0        0.0        0.0
1   20   20       init            0          2   0.5        0.0        0.5
2   26   16   shift-10            2          1   0.5        0.0        0.5
3   32   32       init            0          2   0.5        0.0        0.5
4   60   60       stop            3          0   0.0        1.0        0.0
5   80   80       stop            2          0   0.5        0.5        0.0
6  100  100  end-scan0            0         -1   0.0        1.0        0.0
7  100  100  end-tran1            1         -1   0.0        1.0        0.0
8  100  100  end-tran2            2         -1   0.0        1.0        0.0
9  100  100  end-tran3            3         -1   0.0        1.0        0.0
move_events
    pos             type  start_frame  end_frame  prob  drop_prob  cont_prob  \
0     1             load           -1          0   1.0        0.0        0.

In [57]:

graph_map = ribograph(node_map=nodemap)
print('graph map')
print(graph_map)


    pos             type  start_frame  end_frame  prob  drop_prob  cont_prob  \
0     1             load           -1          0   1.0        0.0        0.0   
1   100        end-scan0            0         -1   0.0        1.0        0.0   
2   100        end-tran1            1         -1   0.0        1.0        0.0   
3   100        end-tran2            2         -1   0.0        1.0        0.0   
4   100        end-tran3            3         -1   0.0        1.0        0.0   
5    20       init/begin            0          2   0.5        0.0        0.5   
6    26   shift-10/begin            2          1   0.5        0.0        0.5   
7    32       init/begin            0          2   0.5        0.0        0.5   
8    60       stop/begin            3          0   0.0        1.0        0.0   
9    80       stop/begin            2          0   0.5        0.5        0.0   
10   20      init/finish            2          2   1.0        0.0        1.0   
11   16  shift-10/finish            1   

In [58]:

edgelist = table2edgelist(graph_map)
print('edgelist')
print(edgelist)


edgelist
        node    target  weight  state
0    (20, 0)   (20, 2)     0.5   init
1    (26, 2)  (100, 1)     0.5  shift
2    (32, 0)   (32, 2)     0.5   init
3    (80, 2)   (80, 0)     0.5   rein
4    (20, 2)   (26, 2)     1.0  tran2
5    (32, 2)   (80, 2)     1.0  tran2
6    (80, 0)  (100, 0)     1.0   scan
7    (1, -1)   (20, 0)     1.0   scan
8   (100, 0)  (-1, -1)     1.0   drop
9   (100, 1)  (-1, -1)     1.0   drop
10   (80, 2)  (-1, -1)     0.5   drop
11   (20, 0)   (32, 0)     0.5   scan
12   (26, 2)   (32, 2)     0.5  tran2
13   (32, 0)   (80, 0)     0.5   scan


In [59]:
prob_graph = nx.from_pandas_edgelist(edgelist, 'node', 'target', 'weight', create_using=nx.DiGraph)

ribopaths = get_ribopaths(prob_graph, graph_map)
print('ribopaths')
print(ribopaths)

ribopaths
[[((1, -1), (20, 0)), ((20, 0), (20, 2)), ((20, 2), (26, 2)), ((26, 2), (100, 1)), ((100, 1), (-1, -1))], [((1, -1), (20, 0)), ((20, 0), (20, 2)), ((20, 2), (26, 2)), ((26, 2), (32, 2)), ((32, 2), (80, 2)), ((80, 2), (80, 0)), ((80, 0), (100, 0)), ((100, 0), (-1, -1))], [((1, -1), (20, 0)), ((20, 0), (20, 2)), ((20, 2), (26, 2)), ((26, 2), (32, 2)), ((32, 2), (80, 2)), ((80, 2), (-1, -1))], [((1, -1), (20, 0)), ((20, 0), (32, 0)), ((32, 0), (32, 2)), ((32, 2), (80, 2)), ((80, 2), (80, 0)), ((80, 0), (100, 0)), ((100, 0), (-1, -1))], [((1, -1), (20, 0)), ((20, 0), (32, 0)), ((32, 0), (32, 2)), ((32, 2), (80, 2)), ((80, 2), (-1, -1))], [((1, -1), (20, 0)), ((20, 0), (32, 0)), ((32, 0), (80, 0)), ((80, 0), (100, 0)), ((100, 0), (-1, -1))]]


In [60]:
flux_edges = calc_flux(prob_graph, edgelist, ribopaths)
flux_edges.columns = ['source', 'target', 'weight', 'state', 'flux']
flux_graph = nx.from_pandas_edgelist(flux_edges, 'source', 'target', ['state', 'weight', 'flux'], create_using=nx.DiGraph)

print('flux_edges')
print(flux_edges)

flux_edges.to_csv('graph.csv', index=False, sep='\t')


flux_edges
      source    target  weight  state  flux
0    (20, 0)   (20, 2)     0.5   init  0.50
1    (26, 2)  (100, 1)     0.5  shift  0.25
2    (32, 0)   (32, 2)     0.5   init  0.25
3    (80, 2)   (80, 0)     0.5   rein  0.25
4    (20, 2)   (26, 2)     1.0  tran2  0.50
5    (32, 2)   (80, 2)     1.0  tran2  0.50
6    (80, 0)  (100, 0)     1.0   scan  0.50
7    (1, -1)   (20, 0)     1.0   scan  1.00
8   (100, 0)  (-1, -1)     1.0   drop  0.50
9   (100, 1)  (-1, -1)     1.0   drop  0.25
10   (80, 2)  (-1, -1)     0.5   drop  0.25
11   (20, 0)   (32, 0)     0.5   scan  0.50
12   (26, 2)   (32, 2)     0.5  tran2  0.25
13   (32, 0)   (80, 0)     0.5   scan  0.25
